In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import joblib





In [5]:
df=pd.read_csv("household_power_consumption.csv\household_power_consumption.csv")
df

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
1,16/12/2006,17:25:00,5.36,0.436,233.63,23,0,1,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23,0,2,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23,0,1,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0,1,17.0
...,...,...,...,...,...,...,...,...,...
1048570,13/12/2008,21:34:00,0.426,0.076,242.27,1.8,0,0,0.0
1048571,13/12/2008,21:35:00,0.424,0.076,242.1,1.8,0,0,0.0
1048572,13/12/2008,21:36:00,0.422,0.076,241.73,1.8,0,0,0.0
1048573,13/12/2008,21:37:00,0.422,0.078,242.56,1.8,0,0,0.0


In [6]:
df=pd.read_csv("household_power_consumption.csv\household_power_consumption.csv",
               sep=",",
               parse_dates={'DateTime':['Date','Time']},
               infer_datetime_format=True,
               low_memory=False,
               na_values=['?'])
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')

df=df.set_index('DateTime')

cols = ['Global_active_power','Global_reactive_power','Voltage',
        'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

C:\Users\irams\AppData\Local\Temp\ipykernel_22372\556710515.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df=pd.read_csv("household_power_consumption.csv\household_power_consumption.csv",
C:\Users\irams\AppData\Local\Temp\ipykernel_22372\556710515.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df=pd.read_csv("household_power_consumption.csv\household_power_consumption.csv",
C:\Users\irams\AppData\Local\Temp\ipykernel_22372\556710515.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df=pd.read_csv("household_power_co

In [7]:
df['Avg_Intensity_1h'] = df['Global_intensity'].rolling(window=60).mean()
df['Avg_Intensity_1d'] = df['Global_intensity'].rolling(window=1440).mean()

In [8]:
df.fillna(method="bfill", inplace=True)
df

C:\Users\irams\AppData\Local\Temp\ipykernel_22372\3840312381.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Avg_Intensity_1h,Avg_Intensity_1d
DateTime,,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,17.613333,10.592500
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,17.613333,10.592500
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,17.613333,10.592500
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,17.613333,10.592500
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,17.613333,10.592500
...,...,...,...,...,...,...,...,...,...
2008-12-13 21:34:00,0.426,0.076,242.27,1.8,0.0,0.0,0.0,1.766667,5.648056
2008-12-13 21:35:00,0.424,0.076,242.10,1.8,0.0,0.0,0.0,1.740000,5.644722
2008-12-13 21:36:00,0.422,0.076,241.73,1.8,0.0,0.0,0.0,1.743333,5.641389


In [9]:
X = df[['Global_reactive_power','Voltage','Global_intensity',
        'Sub_metering_1','Sub_metering_2','Sub_metering_3',
        'Avg_Intensity_1h','Avg_Intensity_1d']]   # Features

y = df['Global_active_power']  # Target

In [10]:
X_train,X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42, shuffle=False)

In [11]:
#Linear Regression
lr_model=LinearRegression()
lr_model.fit(X_train,y_train)

y_pred_lr=lr_model.predict(X_test)

print("Linear Regression R²:", r2_score(y_test, y_pred_lr))
print("Linear Regression MSE:", mean_squared_error(y_test, y_pred_lr))
print("Linear Regression MAE:", mean_absolute_error(y_test, y_pred_lr))
print("Linear Regression RMSE:", root_mean_squared_error(y_test, y_pred_lr))

Linear Regression R²: 0.9986722816492729
Linear Regression MSE: 0.001414388962418582
Linear Regression MAE: 0.026147572564093148
Linear Regression RMSE: 0.03760836293191425


In [12]:
# rf_model=RandomForestRegressor(n_estimators=100,random_state=42)
# rf_model.fit(X_train,y_train)
# y_pred_rf=rf_model.predict(X_test)

# print("RandomForestRegressor R²:", r2_score(y_test, y_pred_rf))
# print("RandomForestRegressor MSE:", mean_squared_error(y_test, y_pred_rf))
# print("RandomForestRegressor MAE:", mean_absolute_error(y_test, y_pred_rf))
# print("RandomForestRegressor RMSE:", root_mean_squared_error(y_test, y_pred_rf))

In [13]:
xgb_model = XGBRegressor(
    n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42
)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("XGBRegressor R²:", r2_score(y_test, y_pred_xgb))
print("XGBRegressor MSE:", mean_squared_error(y_test, y_pred_xgb))
print("XGBRegressor MAE:", mean_absolute_error(y_test, y_pred_xgb))
print("XGBRegressor RMSE:", root_mean_squared_error(y_test, y_pred_xgb))

XGBRegressor R²: 0.9991006509300099
XGBRegressor MSE: 0.0009580566520443251
XGBRegressor MAE: 0.02126391801707004
XGBRegressor RMSE: 0.030952490239790482


In [14]:
best_model=xgb_model
# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7]
}
# Run grid search
grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=3, scoring='r2')
grid_search.fit(X_train, y_train)

# Best model is the best_estimator_
best_model = grid_search.best_estimator_


In [15]:
joblib.dump(best_model, "energy_model.pkl")


['energy_model.pkl']

In [19]:
# Load the trained model
loaded_model = joblib.load("energy_model.pkl")

# Predict on new data
predictions = loaded_model.predict(X_test)
predictions


array([0.14106286, 0.14060861, 0.12505485, ..., 0.4082155 , 0.40823832,
       0.40823832], shape=(209715,), dtype=float32)